In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='f395e6b5-71c4-4a62-81be-bac90e1b0691', created_at=datetime.datetime(2025, 8, 19, 17, 29, 32, 608239, tzinfo=TzInfo(UTC)), name='2bc298b3-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 19, 17, 29, 32, 608239, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='3fe394e4-572b-4a00-9f22-a20bfed84cbd', agent_id='a1abb90e-c673-4448-a4e2-841170568840', sequence_id=915, task_id='f395e6b5-71c4-4a62-81be-bac90e1b0691', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 19, 17, 29, 32, 641707, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)

╭───────────────────────── USER [08/19/2025 17:29:32] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 17:29:38] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Gathering AI news information                                                │
│                                                                              │
│ The user asked for the latest on AI and AI startups, and I see this is a     │
│ timely request! I need to use the web search tool to gather the latest news. │
│ I’ll set up a query like “latest AI news” or “AI startups funding August     │
│ 2025,” specifically for the current date, August 19, 2025. I’ll keep my      │
│ search broad to cover trends, funding, layoffs, regulation, and major        │
│ product launches. I’ll ensure the search results stay accurate without       │
│ guessing.                                                                    │
│                                                                              │
│ Conducting web search for AI news                                            │
│                                                                              │
│ I’m planning to run a web search for “AI startups funding August 2025 latest │
│ news” and perhaps another for “major AI companies announcements August       │
│ 2025.” It might be more efficient to use one broad query to gather multiple  │
│ sources at once. I’ll just need to call the web_search function to get the   │
│ most relevant results. This should help me find the latest happenings in the │
│ AI sector effectively!                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 17:29:44] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest AI news and AI startups August 2025 \u2014 funding       │
│  rounds, layoffs, product launches, regulation, acquisitions, major compani  │
│  (OpenAI, Anthropic, Google, Microsoft, Nvidia) and notable startups",       │
│    "model": "gpt-4o-mini",                                                   │
│    "search_context_size": "high"                                             │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 17:30:17] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "As of August 19, 2025, the artificial intelligence (AI) sector   │
│  has experienced significant developments across funding, product launches,  │
│  regulatory changes, and corporate activities. Here's an overview of the     │
│  latest AI news:\n\n**Funding and Investments**\n\n- **OpenAI**: In August   │
│  2025, OpenAI secured $8.3 billion in funding, elevating its valuation to    │
│  $300 billion. This oversubscribed round was led by Dragoneer Investment     │
│  Group, with participation from Blackstone, TPG, and Andreessen Horowitz. T  │
│  funds are intended to bolster OpenAI's AI infrastructure and research       │
│  initiatives.                                                                │
│  ([fourweekmba.com](https://fourweekmba.com/top-ai-business-stories-august-  │
│  -2025/?utm_source=openai))\n\n- **Anthropic**: The AI research company      │
│  Anthropic is reportedly raising approximately $5 billion at a valuation of  │
│  $170 billion. This funding round is five times oversubscribed, indicating   │
│  strong investor confidence in Anthropic's AI models and research.           │
│  ([ts2.tech](https://ts2.tech/en/billion-dollar-bids-breakthroughs-backlash  │
│  i-news-roundup-aug-18-19-2025/?utm_source=openai))\n\n- **Cohere**:         │
│  Toronto-based AI startup Cohere closed a $500 million Series D funding      │
│  round, valuing the company at $6.8 billion. Investors include tech giants'  │
│  funds such as Radical Ventures, Nvidia, and Salesforce. Cohere focuses on   │
│  enterprise AI models and plans to use the funds to expand globally and      │
│  develop secure AI solutions for businesses and governments.                 │
│  ([ts2.tech](https://ts2.tech/en/billion-dollar-bids-breakthroughs-backlash  │
│  i-news-roundup-aug-18-19-2025/?utm_source=openai))\n\n**Product Launches a  │
│  Innovations**\n\n- **OpenAI's GPT-5**: OpenAI unveiled GPT-5 on August 7,   │
│  2025. The model demonstrates significant technical breakthroughs in         │
│  reasoning and multimodal capabilities, introducing tiered pricing to enhan  │
│  accessibility. Early benchmarks show a 40% improvement over GPT-4 in compl  │
│  task handling, with enterprise integrations already rolling out globally.   │
│  ([medium.com](https://medium.com/%40CherryZhouTech/ai-news-august-2-8-2025  │
│  op-10-ai-innovations-driving-the-future-a8da254a04f9?utm_source=openai))\n  │
│  - **Microsoft's Windows 11 AI Features**: Microsoft is rolling out several  │
│  new AI-powered features and user interface changes to Windows 11. Key       │
│  enhancements include Copilot Vision, an AI tool integrated with the Copilo  │
│  assistant for real-time interaction with desktop content, and an AI Agent   │
│  Settings, allowing users to make changes by typing requests like \u201cmak  │
│  my cursor larger.\u201d These updates aim to enhance usability, performanc  │
│  and AI integration for Windows 11 users.                                    │
│  ([windowscentral.com](https://www.windowscentral.com/microsoft/windows-11/  │
│  p-6-new-features-and-changes-coming-to-windows-11-in-august-2025-from-ai-a  │
│  nts-to-improved-bsod-screens?utm_source=openai))\n\n**Regulatory and Polic  │
│  Developments**\n\n- **U.S. Federal AI Regulation**: The U.S. federal        │
│  government has enacted a 10-y

╭──────────────────────── AGENT [08/19/2025 17:30:21] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Summarizing news trends                                                      │
│                                                                              │
│ Okay, web search has given me a summary with sources covering the latest     │
│ news. I need to focus on the key points like funding rounds (OpenAI at       │
│ $8.3B, Anthropic at $5B, and Cohere at $500M), new product launches (GPT-5,  │
│ Windows 11 AI), regulatory changes, and acquisitions.                        │
│                                                                              │
│ I'll also comment on the implications for startups, mentioning the           │
│ competitive funding environment, high valuations, and trends in AI. I should │
│ provide options for the user to request details on specific topics!          │
│                                                                              │
│ Preparing the final answer                                                   │
│                                                                              │
│ I should avoid heavy formatting, but using bullet points is acceptable. I’ll │
│ offer to fetch full articles or the latest funding list too. It's important  │
│ to include context — so I'll state "as of August 19, 2025."                  │
│                                                                              │
│ I’ll make sure to cite the sources and possibly include URLs as              │
│ parenthetical citations. My goal is to create a straightforward,             │
│ easy-to-read answer in plain text that effectively communicates the          │
│ information to the user!                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 17:30:25] ─────────────────────────╮
│ Here’s a concise roundup of the latest AI and AI-startup news (as of Aug 19, │
│ 2025), with the key takeaways and what they mean for founders and investors. │
│                                                                              │
│ Top headlines                                                                │
│                                                                              │
│  • Big funding rounds and high valuations: OpenAI reportedly raised about    │
│    $8.3B (valuation ≈ $300B) and Anthropic is raising roughly $5B (≈ $170B). │
│    Cohere closed a $500M Series D (~$6.8B valuation). (sources: FourWeekMBA  │
│    / TS2 Tech)                                                               │
│  • Major product launches: OpenAI unveiled GPT‑5 with stronger reasoning and │
│    multimodal capabilities; Microsoft is rolling more Windows 11 AI features │
│    (Copilot Vision, AI Agents). (sources: Medium, Windows Central)           │
│  • Regulation and policy movement: U.S. federal action is blocking           │
│    state-level AI regulation for a 10‑year period to create a unified        │
│    federal approach; China is increasing formal AI safety measures and       │
│    standards. (sources: TechRadar, Time)                                     │
│  • M&A and infrastructure deals: CoreWeave (Nvidia-backed) acquired Weights  │
│    & Biases; AMD made acquisitions in silicon‑photonics/related areas to     │
│    bolster AI hardware capabilities. (sources: Wikipedia/press summaries,    │
│    LinkedIn post)                                                            │
│  • Industry trends and workforce: Widespread adoption of AI agents in game   │
│    development (≈87% of developers using AI agents); persistent AI talent    │
│    shortage and rising salary pressure. (sources: Reuters, ITPro)            │
│                                                                              │
│ What this means for startups and investors                                   │
│                                                                              │
│  • Capital is still plentiful at the top end (large checks and huge          │
│    valuations), but competition for distribution and enterprise customers    │
│    will be intense.                                                          │
│  • Infrastructure and tooling remain attractive (GPU/cloud, model            │
│    management, observability, fine-tuning, inference-optimized stacks).      │
│    Investors and acquirers are backing these areas.                          │
│  • Regulatory risk is rising: expect more compliance, safety and audit       │
│    requirements (and opportunities for startups that help with governance,   │
│    explainability and risk monitoring).                                      │
│  • Talent is a bottleneck — hiring, retention, remote hiring, training       │
│    programs, and tools that let non-experts build/use models are strategic   │
│    advantages.                                                               │
│  • M&A is active: specialized tooling and infra companies are prime          │
│    acquisition targets for cloud and chip players.                           │
│                                                                              │
│ Practical suggestions for founders right now                                 │
│                                                                              │
│  • Prioritize product-market fit and capital efficiency; large model play    │
│    with weak monetization is risky despite abundant capital.                 │
│  • Build governance and safety capabilities early — regulatory scrutiny is   │
│    increasing globally.                                                      │
│  • Consider partnerships (cloud, hardware vendors, enterprise ISVs) to       │
│    overcome distribution and 

Streaming timed out after 60 seconds - returning collected messages
